# Create Environment

## Import 

In [1]:
import itertools
import pandas as pd
import geopandas as geopd

import numpy as np
from sklearn.model_selection import train_test_split

import gym
from gym.wrappers import TimeLimit
# import time

import warnings
warnings.filterwarnings('ignore')


In [2]:
import ray
# import tune
from ray import tune
from ray.rllib.agents import ppo, a3c, dqn
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env


In [3]:
ray.init()


{'node_ip_address': '127.0.0.1',
 'raylet_ip_address': '127.0.0.1',
 'redis_address': '127.0.0.1:45117',
 'object_store_address': '/tmp/ray/session_2022-02-21_12-47-59_948171_25330/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-02-21_12-47-59_948171_25330/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-02-21_12-47-59_948171_25330',
 'metrics_export_port': 63412,
 'gcs_address': '127.0.0.1:56266',
 'node_id': 'd553d051afe8bc323e6af46ae0ac10e76e3571942174f1b8d87b7ffb'}

## 1. Get the data

In [4]:
## import data
nb_of_data = 'Manhattan'

data = pd.read_csv(f'./Data/demand_{nb_of_data}.csv')
distance_data = pd.read_csv(f'./Data/distance_info_{nb_of_data}.csv')
# distance_data = geopd.GeoDataFrame(pd.read_csv(f'./Data/distance_info_{nb_of_data}.csv'))


In [5]:
# Env params:
max_episode_steps = 300
max_client = 30

env_config = {'demand_data': data,
              'distance_data': distance_data,
              'client_limit': max_client,
              'customers_per_taxi': 3,
              'max_timestep': 250}
del data, distance_data


Add pick up action
multiple customer in taxi instead of 1 (car sharing idea)
Drop odd action????
add time instead of limited number of customers

## 2. Define the environment

In [6]:
class Multi_passanger_taxi(gym.Env):
    def __init__(self, env_config):
        # Get data from input
        self.data = env_config['demand_data']
        self.distance_data = env_config['distance_data']
        self.max_timestep = env_config['max_timestep']
        # max number of customer per taxi
        self.n = env_config['customers_per_taxi']

        self.locations = np.unique(self.distance_data.PULocationID.values)
        self.nb_locations = len(self.locations)

        self.locations = dict(zip(range(self.nb_locations), self.locations))
        self.location_to_index = dict((v, k)
                                      for k, v in self.locations.items())

        self.customers = 0
        self.custumer_count = self.data.Demand.sum()
        self.customers_limit = min(
            self.custumer_count, env_config['client_limit'])
        self.current_client_count = 0

        self.action_space = gym.spaces.Dict({'move': gym.spaces.Discrete(self.nb_locations),
                                            'pickup': gym.spaces.Discrete(2)})

        # Define Observation space
        spaces = {'position': gym.spaces.Box(low=np.array([0]), high=np.array([self.nb_locations-1]), dtype=int),
                  'state': gym.spaces.Box(low=-1, high=self.nb_locations-1, shape=(self.n,), dtype=int), }

        self.observation_space = gym.spaces.Dict(spaces)
        # ## Initialise the space
        self.reset()

    def get_distane(self, PU_location, DO_location):
        distance = self.distance_data[(self.distance_data.PULocationID == PU_location) &
                                      (self.distance_data.DOLocationID == DO_location)]['distance'].values
        return distance[0]

    def step(self, action):
        # print('*********************************next step*********************************')
        done = False
        reward = 0
        move_action = action['move']
        pick_up_action = action['pickup']
        taxi_location = self.locations[self.state['position'][0]]
        taxi_state = self.state['state']
        # print(f'pick up action is {pick_up_action}')
        if pick_up_action == 1:  # If action is pick up client:
            # print(taxi_state.values())
            if -1 not in taxi_state:  # taxi is full
                # print(f'We are trying to pick up but full taxi {taxi_state}')
                reward -= 100
                # print(self.state)
            else:  # taxi has an empty spot
                # print('We are trying to pick up someone')
                for i in range(self.n):
                    if taxi_state[i] == -1:
                        if taxi_location in self.demand_dict.keys():  # Current location has a client:
                            # print(f'Pick up client in position {i}')
                            self.current_client_count += 1
                            destination_location = self.demand_dict[taxi_location].pop(
                                0)
                            self.state['state'][i] = self.location_to_index[destination_location]
                            if self.demand_dict[taxi_location] == []:
                                self.demand_dict.pop(taxi_location)
                            reward += 10
                            break

        action_location = self.locations[move_action]
        if action_location in taxi_state:  # if action is going to the client destination
            destination_indexes, = np.where(taxi_state == action_location)
            # print(f'Dropped off client {destination_indexes}')
            # print('npwhere res',destination_indexes)
            for destination_index in destination_indexes:
                # print('single index', destination_index)
                # print('chennnn')
                self.customers += 1
                self.current_client_count -= 1
                self.state['state'][destination_index] = -1
                reward += self.get_distane(taxi_location, action_location)*100
        else:  # We have a client but we are not going to his destination
            if self.current_client_count == 0:
                # print(f'Wrong location with neg rew {taxi_state}')
                reward += -self.get_distane(taxi_location, action_location)*10
            # else:
                # print(f'Wrong location but no client {taxi_state}')

        self.taxi_path.append(move_action)
        self.state['position'][0] = move_action
        self.time_step += 1

        if self.customers == self.customers_limit or len(self.demand_dict) == 0:
            done = True

        # Set placeholder for info
        info = {'taxi location & state': self.state,
                'nb of satistied customers ': self.customers,
                'number of steps': self.time_step
                }

        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass

    def reset(self):
        self.time_step = 0
        self.taxi_path = []
        self.state = self.observation_space.sample()
        self.state['state'] = -np.ones(self.n)
        # print(self.state)
        self.demand_dict = {pu: [] for pu in self.data.PU_LocationID}
        for index in range(len(self.data)):
            row_index = self.data.loc[index]
            # print(row_index.PU_LocationID)
            self.demand_dict[row_index.PU_LocationID] += [
                row_index.DO_LocationID for _ in range(row_index.Demand)]
        self.customers = 0
        return self.state


In [7]:
class Single_passanger_taxi(gym.Env):
    def __init__(self, env_config):
        # Get data from input
        self.data = env_config['demand_data']
        self.distance_data = env_config['distance_data']
        self.max_timestep = env_config['max_timestep']

        self.locations = np.unique(self.distance_data.PULocationID.values)
        self.nb_locations = len(self.locations)

        self.locations = dict(zip(range(self.nb_locations), self.locations))
        self.location_to_index = dict((v, k)
                                      for k, v in self.locations.items())

        self.customers = 0
        self.custumer_count = self.data.Demand.sum()
        self.customers_limit = min(
            self.custumer_count, env_config['client_limit'])
        # Define Actions we can
        self.action_space = gym.spaces.Discrete(self.nb_locations*2)

        # Define Observation space
        self.observation_space = gym.spaces.Box(low=np.array([0, -1]),
                                                high=np.array([self.nb_locations-1, self.nb_locations-1]), dtype=int)  # TODO: maybe repeated info here ????

        # ## Initialise the space
        self.reset()

    def get_distane(self, PU_location, DO_location):
        distance = self.distance_data[(self.distance_data.PULocationID == PU_location) &
                                      (self.distance_data.DOLocationID == DO_location)]['distance'].values
        return distance[0]

    def discretize_action_space(self, action):
        list_action = [(i, j) for i in range(self.nb_locations)
                       for j in [0, 1]]
        dict_action = dict(zip(range(2*self.nb_locations), list_action))
        return dict_action[action]

    def pick_up_action(self):
        taxi_location = self.locations[self.state[0]]
        if self.state[1] == -1:  # Check if taxi is empty
            if taxi_location in self.demand_dict.keys():  # Current location has a client:
                destination_location = self.demand_dict[taxi_location].pop(0)
                self.state[1] = self.location_to_index[destination_location]
                if self.demand_dict[taxi_location] == []:
                    self.demand_dict.pop(taxi_location)
                return 10
            else:  # Taxi empty and location has no client:
                return 0
        else:  # Taxi has a client so it's impossible action to pick up one client
            return -100

    def move_action(self, action):
        action_location = self.locations[action]
        taxi_location = self.locations[self.state[0]]
        if self.state[1] == -1:  # Empty taxi
            # return -self.get_distane(taxi_location,action_location)
            return 0
        else:  # taxi already has a client
            taxi_destination = self.locations[self.state[1]]
            # if action is going to the client destination
            if action == self.state[1]:
                self.customers += 1
                self.state[1] = -1
                return self.get_distane(taxi_location, action_location)*100
            else:  # We have a client but we are not going to his destination
                return -self.get_distane(taxi_location, action_location)*10

    def step(self, action):

        action = self.discretize_action_space(action)
        # print(self.state)
        done = False
        reward = 0
        go_action = action[0]
        pick_up_action = action[1]

        if pick_up_action == 1:  # If action is pick up client:
            reward += self.pick_up_action()
        reward = self.move_action(go_action)

        self.taxi_path.append(action[0])
        self.state[0] = action[0]
        self.time_step += 1

        if self.customers == self.customers_limit or len(self.demand_dict) == 0:
            done = True

        # Set placeholder for info
        info = {}

        # Return step information
        return self.state, reward, done, info

    def render(self):
        # Implement viz
        pass

    def reset(self):
        self.time_step = 0
        self.taxi_path = []
        self.state = self.observation_space.sample()
        # self.state[2]=0
        self.state[1] = -1
        self.demand_dict = {pu: [] for pu in self.data.PU_LocationID}
        for index in range(len(self.data)):
            row_index = self.data.loc[index]
            # print(row_index.PU_LocationID)
            self.demand_dict[row_index.PU_LocationID] += [
                row_index.DO_LocationID for _ in range(row_index.Demand)]
        self.customers = 0
        return self.state


In [8]:
# Registering the env for easier use.
def env_multi_creator(env_config):
    # return an env instance
    return TimeLimit(Multi_passanger_taxi(env_config), env_config['max_timestep'])


register_env("Multi_passanger_env", env_multi_creator)


In [9]:
def env_single_reator(env_config):
    # return an env instance
    return TimeLimit(Single_passanger_taxi(env_config),env_config['max_timestep'])


register_env("Single_passanger_env", env_single_reator)


## Tune Grid_search training

Returns an ExperimentAnalysis object that allows further analysis of the training results and retrieving the checkpoint(s) of the trained agent. It also simplifies saving the trained agent.

In [10]:
chosen_env = 'Multi_passanger_env'
# 0.005, 0.0001, 0.0005

In [15]:
config = {
    "env": chosen_env,
    'env_config': env_config,
    'framework': 'torch',
    'preprocessor_pref': 'deepmind',"vf_clip_param": 400,# t'log_level': 'INFO',
    "lr": tune.grid_search([0.005, 0.0001, 0.0005]),
}
# trainer_config['env_config']=env_config
# trainer_config['framework']='torch'
stop_criteria = {"episode_reward_max": 0,
    "agent_timesteps_total": 400000}


In [16]:
# tune.run() allows setting a custom log directory (other than ``~/ray-results``)
# and automatically saving the trained agent
analysis = ray.tune.run(
    ppo.PPOTrainer,
    config=config,
    # local_dir=log_dir,
    stop=stop_criteria,
    checkpoint_at_end=True)


Trial name,status,loc,lr
PPOTrainer_Multi_passanger_env_29773_00000,PENDING,,0.005
PPOTrainer_Multi_passanger_env_29773_00001,PENDING,,0.0001
PPOTrainer_Multi_passanger_env_29773_00002,PENDING,,0.0005


(PPOTrainer pid=25400) 2022-02-21 12:52:25,638	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=25400) 2022-02-21 12:52:25,639	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(PPOTrainer pid=25399) 2022-02-21 12:52:25,638	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(PPOTrainer pid=25399) 2022-02-21 12:52:25,638	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=25398) 2022-02-21 12:52:33,108	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.rem

Trial name,status,loc,lr
PPOTrainer_Multi_passanger_env_29773_00000,RUNNING,127.0.0.1:25399,0.005
PPOTrainer_Multi_passanger_env_29773_00001,RUNNING,127.0.0.1:25400,0.0001
PPOTrainer_Multi_passanger_env_29773_00002,PENDING,,0.0005



(RolloutWorker pid=25405) 2022-02-21 12:52:34,610	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!


Trial name,status,loc,lr
PPOTrainer_Multi_passanger_env_29773_00000,RUNNING,127.0.0.1:25399,0.005
PPOTrainer_Multi_passanger_env_29773_00001,RUNNING,127.0.0.1:25400,0.0001
PPOTrainer_Multi_passanger_env_29773_00002,PENDING,,0.0005


...

Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_Multi_passanger_env_29773_00002,RUNNING,127.0.0.1:25681,0.0005,55,1309.59,220000,-754.899,-70,-6770,250
PPOTrainer_Multi_passanger_env_29773_00000,TERMINATED,127.0.0.1:25399,0.005,18,482.663,72000,-961.03,30,-2170,250
PPOTrainer_Multi_passanger_env_29773_00001,TERMINATED,127.0.0.1:25400,0.0001,40,1028.81,160000,-744.458,54.8084,-7156.57,250


Result for PPOTrainer_Multi_passanger_env_29773_00002:
  agent_timesteps_total: 224000
  custom_metrics: {}
  date: 2022-02-21_13-23-04
  done: true
  episode_len_mean: 250.0
  episode_media: {}
  episode_reward_max: 30.0
  episode_reward_mean: -666.2275985470728
  episode_reward_min: -6770.0
  episodes_this_iter: 16
  episodes_total: 896
  experiment_id: 3c9a11d04f3447e996eb7e80c237bdf2
  hostname: Wyames-MBP.attlocal.net
  info:
    learner:
      default_policy:
        custom_metrics: {}
        learner_stats:
          allreduce_latency: 0.0
          cur_kl_coeff: 2.2781249999999993
          cur_lr: 0.0005000000000000002
          entropy: 3.460679397531735
          entropy_coeff: 0.0
          kl: 0.01127065493350863
          policy_loss: -0.06947654788131996
          total_loss: 38959.793184801325
          vf_explained_var: -0.22896251466966444
          vf_loss: 38959.837273841775
        model: {}
    num_agent_steps_sampled: 224000
    num_agent_steps_trained: 224000
  

(PPOTrainer pid=25681) 2022-02-21 13:23:05,019	WARNING deprecation.py:45 -- DeprecationWarning: `clear_buffer` has been deprecated. Use `Filter.reset_buffer()` instead. This will raise an error in the future!


Trial name,status,loc,lr,iter,total time (s),ts,reward,episode_reward_max,episode_reward_min,episode_len_mean
PPOTrainer_Multi_passanger_env_29773_00000,TERMINATED,127.0.0.1:25399,0.005,18,482.663,72000,-961.03,30,-2170,250
PPOTrainer_Multi_passanger_env_29773_00001,TERMINATED,127.0.0.1:25400,0.0001,40,1028.81,160000,-744.458,54.8084,-7156.57,250
PPOTrainer_Multi_passanger_env_29773_00002,TERMINATED,127.0.0.1:25681,0.0005,56,1332.87,224000,-666.228,30,-6770,250


2022-02-21 13:23:05,473	INFO tune.py:636 -- Total run time: 1845.11 seconds (1844.65 seconds for the tuning loop).


In [17]:
# list of lists: one list per checkpoint; each checkpoint list contains
# 1st the path, 2nd the metric value
checkpoints = analysis.get_trial_checkpoints_paths(
    trial=analysis.get_best_trial("episode_reward_mean", mode='max'),
    metric="episode_reward_mean")


In [18]:
agent = ppo.PPOTrainer(
    config={'env_config': env_config, 'framework': 'torch'}, env=chosen_env)
agent.restore(checkpoints[0][0])


2022-02-21 13:23:05,830	INFO ppo.py:249 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-02-21 13:23:05,831	INFO trainer.py:790 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(RolloutWorker pid=26002) 2022-02-21 13:23:12,418	WARNING deprecation.py:45 -- DeprecationWarning: `rllib.env.remote_vector_env.RemoteVectorEnv` has been deprecated. Use `ray.rllib.env.remote_base_env.RemoteBaseEnv` instead. This will raise an error in the future!
2022-02-21 13:23:12,542	WARNING util.py:55 -- Install gputil for GPU system monitoring.
2022-02-21 13:23:12,563	INFO trainable.py:472 -- Restored on 127.0.0.1 from checkpoint: /Users/wyamebenslimane/ray_results/PPOTrainer_2022-02-21_12-52-20/PPOTrainer_Multi_passanger_env_29773_00002_2_lr=0.0005_2022-02-21_12-52-22/checkpoint_000056/checkpoint-56
2022-02-21 13:23:12,578	IN

In [20]:
env = env_multi_creator(env_config)
to_print = True
num_episodes = 10

env.reset()
total_reward = []
box_flag = isinstance(env.observation_space, gym.spaces.Box)
for episode in range(num_episodes):
    env.reset()
    state = env.reset()
    state_0 = state.copy()
    done = False
    ep_reward = 0
    step = 0
    if to_print:
        print("****************************************************")
        print("EPISODE ", episode)
        # print('Initial state', state_0)

    while not done:
        # print(state)
        action = agent.compute_action(state)
        new_state, reward, done, info = env.step(action)

        ep_reward += reward
        step += 1

        if done:
            total_reward.append(ep_reward)
            if to_print:
                # Here, we decide to only print the last state (to see if our agent is on the goal or fall into an hole)
                # num_transported= list(state_0[1:]).count(1) - list(new_state[1:]).count(1)
                print('Initial state : ', state_0,
                      ' Final state : ', new_state)

                # We print the number of step it took.
                print(
                    f"Number of steps {step}, customers transported are {env.customers}, the total reward is {ep_reward}")

            break
        state = new_state
        # state = tuple(new_state) if box_flag else new_state

if to_print:
    print('Mean score: %.3f of %i games!' %
          (np.mean(total_reward), num_episodes))
# return np.mean(total_reward)


2022-02-21 13:56:29,667	WARNING deprecation.py:45 -- DeprecationWarning: `compute_action` has been deprecated. Use `Trainer.compute_single_action()` instead. This will raise an error in the future!


****************************************************
EPISODE  0
Initial state :  OrderedDict([('position', array([54])), ('state', array([2., 2., 2.]))])  Final state :  OrderedDict([('position', array([54])), ('state', array([2., 2., 2.]))])
Number of steps 250, customers transported are 1, the total reward is -261.886700608727
****************************************************
EPISODE  1
Initial state :  OrderedDict([('position', array([8])), ('state', array([2., 2., 2.]))])  Final state :  OrderedDict([('position', array([8])), ('state', array([2., 2., 2.]))])
Number of steps 250, customers transported are 0, the total reward is 30
****************************************************
EPISODE  2
Initial state :  OrderedDict([('position', array([48])), ('state', array([2., 2., 2.]))])  Final state :  OrderedDict([('position', array([48])), ('state', array([2., 2., 2.]))])
Number of steps 250, customers transported are 0, the total reward is 30
***************************************